In [1]:
import numpy as np
from scipy.stats import bernoulli
import matplotlib.pyplot as plt



In [2]:
def gen_data(theta, n, m):
    X = np.random.randn(n, m + 1)
    X[:, 0] = 1  # first column set to 1
    beta = np.random.randn(m + 1, 1)  # random coefficients
    bern = np.transpose(bernoulli.rvs(size=n, p=theta))  # Bernoulli distribution
    Y = 1 / (1 + np.exp(-(np.matmul(X, beta))))  # Logistic function
    
    for i in range(len(Y)):  # classification of Y
        if Y[i] > 0.5:
            Y[i] = 1
        else:
            Y[i] = 0
    
    for i in range(len(Y)):  # flipping according to Bernoulli distribution
        if bern[i] == 1:
            Y[i] = 1 - Y[i]
    
    return X, beta, Y

\begin{align*}
\text{Log Likelihood Function:} \\
\text{logC} &= \sum_{n=1}^{N} \left( y_n \log(\sigma(\beta^T X_n)) + (1-y_n) \log(1-\sigma(\beta^T X_n)) \right) \\
\text{Gradient:} \\
\frac{\partial \text{logC}}{\partial \beta} &= X^T (Y - Y_{\text{predicted}})
\end{align*}



In [7]:
def logistic_regression(X, Y, k, tau, lam):
    z = float(len(X))
    b = np.random.randn(len(X[0]), 1)  # random initialization of the parameters
    previous_cost = 0  # initial cost set to 0
    cost = np.array([])

    for i in range(k):  # k is number of iterations
        y_pred = 1.0 / (1 + np.exp(-(np.matmul(X, b))))  # predicted y
        temp = y_pred - Y  # Y - y_pred
        temp2 = np.ones_like(Y) - Y
        
        # Cost function / Cross Entropy
        current_cost = -(1/z) * np.matmul(np.transpose(Y), np.log(y_pred, out=np.zeros_like(y_pred), where=(y_pred != 0))) - \
                       np.matmul(np.transpose(temp2), np.log(temp2, out=np.zeros_like(temp2), where=(temp2 != 0)))
        
        cost = np.append(cost, current_cost)
        
        if abs(current_cost - previous_cost) <= tau:  # tau - threshold condition on change in cost function
            break
        
        previous_cost = current_cost
        grad = (1 / z) * np.matmul(np.transpose(X), temp)  # gradient calculation
        b = b - lam * grad  # improved b, lam is step size / learning parameter
    
    return y_pred, cost[-1]


In [10]:
import numpy as np

def logistic_regression_L1(X, Y, k, tau, lam, alpha):
    z = float(len(X))
    b = np.random.randn(len(X[0]), 1)  # random initialization of the parameters
    previous_cost = 0  # initial cost set to 0
    cost = np.array([])
    b_L2 = 0
    
    for i in range(1, len(b)):
        b_L2 += abs(b[i])  # L1 norm
    
    I_L2 = np.ones((len(b), 1))
    I_L2[:, 0] = 0  # 'I' matrix
    I_L2 = I_L2 * np.sign(b)
    
    for i in range(k):  # k is number of iterations
        y_pred = 1.0 / (1 + np.exp(-(np.matmul(X, b))))  # predicted y
        temp = y_pred - Y  # Y - y_pred
        temp2 = np.ones_like(Y) - Y
        
        # Cost / cross entropy calculation
        current_cost = -(1/z) * np.matmul(np.transpose(Y), np.log(y_pred, out=np.zeros_like(y_pred), where=(y_pred != 0))) - \
                       np.matmul(np.transpose(temp2), np.log(temp2, out=np.zeros_like(temp2), where=(temp2 != 0)))
        
        # For L1 regression
        current_cost += (1/z) * (alpha * b_L2)
        
        cost = np.append(cost, current_cost)
        
        if abs(current_cost - previous_cost) <= tau:  # tau - threshold condition on change in cost function
            break
        
        previous_cost = current_cost
        grad = (1 / z) * np.matmul(np.transpose(X), temp)  # gradient calculation
        # Gradient calculation for L1 regression, alpha is tuning parameter
        grad = grad - alpha * I_L2
        b = b - lam * grad  # improved b, lam is step size / learning parameter
    
    return y_pred, cost[-1]


In [11]:
##Ridge regression
def logistic_regression_L2(X, Y, k, tau, lam, alpha):
    z = float(len(X))
    b = np.random.randn(len(X[0]), 1)  # random initialization of the parameters
    previous_cost = 0  # initial cost set to 0
    cost = np.array([])
    iter = []

    b_L2 = 0
    for i in range(1, len(b)):
        b_L2 += b[i] * b[i]  # square of L2 norm

    I_L2 = np.ones((len(b), 1))
    I_L2[:, 0] = 0  # 'I' matrix
    I_L2 = I_L2 * b

    for i in range(k):  # k is number of iterations
        iter.append(i)
        y_pred = 1.0 / (1 + np.exp(-(np.matmul(X, b))))  # predicted y
        temp = y_pred - Y  # Y - y_pred
        temp2 = np.ones_like(Y) - Y

        # Cost / cross entropy calculation
        current_cost = -(1/z) * np.matmul(np.transpose(Y), np.log(y_pred, out=np.zeros_like(y_pred), where=(y_pred != 0))) - \
                       np.matmul(np.transpose(temp2), np.log(temp2, out=np.zeros_like(temp2), where=(temp2 != 0)))
        
        # Cost with L2 term
        current_cost += (1/z) * (alpha * b_L2)
        
        cost = np.append(cost, current_cost)
        
        if abs(current_cost - previous_cost) <= tau:  # tau - threshold condition on change in cost function
            break
        
        previous_cost = current_cost
        grad = (1 / z) * np.matmul(np.transpose(X), temp)  # gradient calculation
        # Gradient for L2 regression, alpha is tuning parameter
        grad = grad - alpha * I_L2
        b = b - lam * grad  # improved b, lam is step size / learning parameter
    
    return y_pred, cost[-1]
